In [20]:
import pandas as pd
import numpy as np

## Read and Prepare Data

In [2]:
df = pd.read_csv("C:/Users/mekho/Documents/PycharmProjects/LinkedIn_Quiz/data/Food/wrangled_data.csv")

C:\Users\mekho\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (6,16,27,28,29,30,37,38,39,43,44,45,99,117,118) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,Unnamed: 0,DISPCODE,SEQNO,STATERE1,SAFETIME,HHADULT,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,...,_PAREC1,_PASTAE1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3,_MMSAWT,MMSANAME,_MMSA
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
names = [str(n).split(',')[0].strip() for n in df.MMSANAME]

df['city'] = names

df = df.loc[df.city != 'nan', :].copy()

In [5]:
df.head()

,Unnamed: 0,DISPCODE,SEQNO,STATERE1,SAFETIME,HHADULT,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,...,_PASTAE1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3,_MMSAWT,MMSANAME,_MMSA,city
512,512,1200.0,2.017000e+09,,1,01,1,88.0,88.0,,...,9.0,9.0,9.0,,,,247.441661,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy
513,513,1200.0,2.017000e+09,,1,02,1,88.0,88.0,,...,1.0,1.0,1.0,,,2,161.334971,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy
514,514,1200.0,2.017000e+09,,1,01,4,88.0,30.0,30,...,1.0,1.0,1.0,,,1,756.699459,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy
515,515,1200.0,2.017000e+09,,1,,2,88.0,5.0,02,...,2.0,1.0,1.0,,,1,1236.323980,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy
516,516,1200.0,2.017000e+09,,1,,4,30.0,10.0,15,...,9.0,9.0,9.0,9,9,,213.968412,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy


In [6]:
df.columns

Index(['Unnamed: 0', 'DISPCODE', 'SEQNO', 'STATERE1', 'SAFETIME', 'HHADULT',
       'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH',
       ...
       '_PASTAE1', '_RFSEAT2', '_RFSEAT3', '_FLSHOT6', '_PNEUMO2', '_AIDTST3',
       '_MMSAWT', 'MMSANAME', '_MMSA', 'city'],
      dtype='object', length=179)

In [7]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
df.head()

,DISPCODE,SEQNO,STATERE1,SAFETIME,HHADULT,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,HLTHPLN1,...,_PASTAE1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3,_MMSAWT,MMSANAME,_MMSA,city
512,1200.0,2.017000e+09,,1,01,1,88.0,88.0,,1.0,...,9.0,9.0,9.0,,,,247.441661,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy
513,1200.0,2.017000e+09,,1,02,1,88.0,88.0,,1.0,...,1.0,1.0,1.0,,,2,161.334971,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy
514,1200.0,2.017000e+09,,1,01,4,88.0,30.0,30,1.0,...,1.0,1.0,1.0,,,1,756.699459,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy
515,1200.0,2.017000e+09,,1,,2,88.0,5.0,02,1.0,...,2.0,1.0,1.0,,,1,1236.323980,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy
516,1200.0,2.017000e+09,,1,,4,30.0,10.0,15,1.0,...,9.0,9.0,9.0,9,9,,213.968412,"Albany-Schenectady-Troy, NY, Metropolitan Stat...",10580.0,Albany-Schenectady-Troy


In [9]:
columns_to_select = [c for c in df.columns if c.startswith('_')]
columns_to_select = ['city'] + columns_to_select
df = df.loc[:, columns_to_select]

In [10]:
df.head()

,city,_URBNRRL,_STSTR,_IMPSEX,_RFHLTH,_PHYS14D,_MENT14D,_HCVU651,_RFHYPE5,_CHOLCH1,...,_PASTRNG,_PAREC1,_PASTAE1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3,_MMSAWT,_MMSA
512,Albany-Schenectady-Troy,3,17069.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,9.0,9.0,9.0,9.0,9.0,,,,247.441661,10580.0
513,Albany-Schenectady-Troy,3,17069.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,,,2,161.334971,10580.0
514,Albany-Schenectady-Troy,3,17069.0,1.0,2.0,1.0,3.0,1.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,,,1,756.699459,10580.0
515,Albany-Schenectady-Troy,3,17069.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,...,2.0,4.0,2.0,1.0,1.0,,,1,1236.323980,10580.0
516,Albany-Schenectady-Troy,3,17069.0,2.0,2.0,3.0,2.0,9.0,1.0,1.0,...,9.0,9.0,9.0,9.0,9.0,9,9,,213.968412,10580.0


In [11]:
df.set_index('city', inplace=True)

In [12]:
df.head()

,_URBNRRL,_STSTR,_IMPSEX,_RFHLTH,_PHYS14D,_MENT14D,_HCVU651,_RFHYPE5,_CHOLCH1,_RFCHOL1,...,_PASTRNG,_PAREC1,_PASTAE1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3,_MMSAWT,_MMSA
city,,,,,,,,,,,,,,,,,,,,,
Albany-Schenectady-Troy,3,17069.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1,...,9.0,9.0,9.0,9.0,9.0,,,,247.441661,10580.0
Albany-Schenectady-Troy,3,17069.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1,...,1.0,1.0,1.0,1.0,1.0,,,2,161.334971,10580.0
Albany-Schenectady-Troy,3,17069.0,1.0,2.0,1.0,3.0,1.0,2.0,1.0,2,...,1.0,1.0,1.0,1.0,1.0,,,1,756.699459,10580.0
Albany-Schenectady-Troy,3,17069.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1,...,2.0,4.0,2.0,1.0,1.0,,,1,1236.323980,10580.0
Albany-Schenectady-Troy,3,17069.0,2.0,2.0,3.0,2.0,9.0,1.0,1.0,1,...,9.0,9.0,9.0,9.0,9.0,9,9,,213.968412,10580.0


## Exploration

In [13]:
dict_col_unique_counts = {}

In [14]:
for c in df.columns:
    print('')
    print(c)
    print(len(df[c].unique()))
    dict_col_unique_counts[c] = len(df[c].unique())
    


_URBNRRL
9

_STSTR
1371

_IMPSEX
2

_RFHLTH
3

_PHYS14D
4

_MENT14D
4

_HCVU651
3

_RFHYPE5
3

_CHOLCH1
4

_RFCHOL1
3

_MICHD
3

_LTASTH1
3

_CASTHM1
3

_ASTHMS1
4

_DRDXAR1
3

_LMTACT1
5

_LMTWRK1
5

_LMTSCL1
6

_PRACE1
19

_MRACE1
19

_HISPANC
3

_RACE
9

_RACEG21
3

_RACEGR3
6

_AGEG5YR
14

_AGE65YR
3

_AGE80
63

_AGE_G
6

_BMI5
3281

_BMI5CAT
5

_RFBMI5
3

_EDUCAG
5

_INCOMG
6

_SMOKER3
5

_RFSMOK3
3

_ECIGSTS
5

_CURECIG
3

_RFBING5
3

_DRNKWEK
235

_RFDRHV5
3

_MISFRT1
3

_MISVEG1
4

_FRTRES1
2

_VEGRES1
2

_FRUTSU1
504

_VEGESU1
1357

_FRTLT1A
3

_VEGLT1A
3

_FRT16A
2

_VEG23A
2

_FRUITE1
3

_VEGETE1
3

_TOTINDA
3

_MINAC11
673

_MINAC21
533

_PACAT1
5

_PAINDX1
3

_PA150R2
4

_PA300R2
4

_PA30021
3

_PASTRNG
3

_PAREC1
5

_PASTAE1
3

_RFSEAT2
3

_RFSEAT3
3

_FLSHOT6
4

_PNEUMO2
4

_AIDTST3
4

_MMSAWT
151809

_MMSA
135


In [15]:
dict_col_unique_counts

{'_URBNRRL': 9,
 '_STSTR': 1371,
 '_IMPSEX': 2,
 '_RFHLTH': 3,
 '_PHYS14D': 4,
 '_MENT14D': 4,
 '_HCVU651': 3,
 '_RFHYPE5': 3,
 '_CHOLCH1': 4,
 '_RFCHOL1': 3,
 '_MICHD': 3,
 '_LTASTH1': 3,
 '_CASTHM1': 3,
 '_ASTHMS1': 4,
 '_DRDXAR1': 3,
 '_LMTACT1': 5,
 '_LMTWRK1': 5,
 '_LMTSCL1': 6,
 '_PRACE1': 19,
 '_MRACE1': 19,
 '_HISPANC': 3,
 '_RACE': 9,
 '_RACEG21': 3,
 '_RACEGR3': 6,
 '_AGEG5YR': 14,
 '_AGE65YR': 3,
 '_AGE80': 63,
 '_AGE_G': 6,
 '_BMI5': 3281,
 '_BMI5CAT': 5,
 '_RFBMI5': 3,
 '_EDUCAG': 5,
 '_INCOMG': 6,
 '_SMOKER3': 5,
 '_RFSMOK3': 3,
 '_ECIGSTS': 5,
 '_CURECIG': 3,
 '_RFBING5': 3,
 '_DRNKWEK': 235,
 '_RFDRHV5': 3,
 '_MISFRT1': 3,
 '_MISVEG1': 4,
 '_FRTRES1': 2,
 '_VEGRES1': 2,
 '_FRUTSU1': 504,
 '_VEGESU1': 1357,
 '_FRTLT1A': 3,
 '_VEGLT1A': 3,
 '_FRT16A': 2,
 '_VEG23A': 2,
 '_FRUITE1': 3,
 '_VEGETE1': 3,
 '_TOTINDA': 3,
 '_MINAC11': 673,
 '_MINAC21': 533,
 '_PACAT1': 5,
 '_PAINDX1': 3,
 '_PA150R2': 4,
 '_PA300R2': 4,
 '_PA30021': 3,
 '_PASTRNG': 3,
 '_PAREC1': 5,
 '_PASTAE1':

## Data Transformation

In [16]:
list_data_transformed = []

In [24]:
def transform_chunk(chunk):
    dict_temp_transformed_data = {}
    print(np.unique(chunk.index)[0])
    dict_temp_transformed_data = np.unique(chunk.index)[0]
    for col in chunk.columns:
        if dict_col_unique_counts[col] < 30:
            print('haha')
            
    return

In [25]:
sth = df.groupby('city').apply(lambda chunk: transform_chunk(chunk))

Albany-Schenectady-Troy
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Albuquerque
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Allentown-Bethlehem-Easton
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Anchorage
haha
haha
haha
haha
haha
ha

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Grand Forks
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Grand Island
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Grand Rapids-Wyoming
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Portland-South Portland
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Portland-Vancouver-Hillsboro
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Providence-Warwick
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
hah

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Toledo
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Topeka
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Tulsa
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
